# VA Passenger Availability SLA Report

Once the upgrade to Linux has been completed it is measured using an automated script on the server measuring the availability of the Passenger Service by checking & logging the availability of the SSID (WLAN) and the software health after the booting phase. The System Uptime and hence the Expected Operation Time starts after the booting phase of the software initiated by power on. Any time the Passenger Service then is not available is considered an Unplanned Downtime.

## Open tasks
- [x] add/improve description (Why? What? How?)
- [x] make consideration of DRM Widevine/Fairplay, Moving Map, Analytics Receiver optional and configurable in the report
- [x] set first appcheck (timestamp) as the start of operation time and power off as the end
- [x] consider CWAP unavailabilities by counting Errors as unavailability
- [x] sum up the 10 minute periods of a failed app check to unavailability time (thus, decrease 'actual operation time' accordingly)
- [x] make the appchecks' downtimes available in df_sla
- [ ] structure the code and, for instance, use functions
- [ ] implement Kibana API interface
- [ ] come up with a CI/CD workflow for this/publish the Jupyter report
- [x] define the overall SLA dataframe
- [x] calculate the Expected Operation Time
- [x] and store it in the SLA dataframe (per day and server)
- [x] sum up the server in-flight reboots as unexpected downtime
- [x] and add them to the SLA dataframe (per day and server)
- [x] clean up the notebook and remove unsused, irrelevant variables
- [x] introduce server/tailsign mapping
- [ ] get the month, which I use to initialise df_SLA, automatically out of the imported data
- [ ] do not consider AppCheck downtimes that happened on ground
- [x] do not consider CWAP Errors to reduce the expected operational time
- [ ] consider the log events before the first and after the last WoW event (where no other corresponding event exists)

In [ ]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import warnings

# Read the log data

## Source of information
The data, that the Juypter Notebook processes and bases the report on, comes from **BoardConnect Kibana Prod**.


Since the Notebook currently does not have a direct interface to Kibana implemented, we need to request the data
in Kibana and export it as a **CSV** file.

In Kibana, one fine a stored request which is named **VAPaxAvailSLAv2** and which should be executed using the
relevant timeframe that should be considered in the report (for instance: November 2021).

In a first step, the CSV file needs to be provided to Juypter Notebook, so that it can be read and processed.

In [ ]:
# read csv (VAPaxAvailSLAv2) into dataframe
df = pd.read_csv('/Users/u293733/git.jupyter-notebooks/VAPaxAvailSLAv2.csv')
# outlign the shape 
df.shape


# Data adjustments

We keep the original data in the dataframe df and create a copy of it for further processing.  
In this copy we adjust the data so that it becomes - for instance - better readable.

## Transfer Event Ids

In the original dataset, the events which had been logged, are stored as numbers.  
We convert these numbers to humanly readable event descriptions. 

|Event ID | Event name|
|--------|--------|
|10891+ | CWAP health |
|10900+ | CWAP status |
|30100 | AppCheck |
|10811 | Weight on wheels |
|10812 | Weight off wheels |

In [ ]:
# create new dataframe from df and add column EventID
df_new = df.copy()
# map the eventID to readable event names
eventId_map = {'eventId': {10891: 'CWAP health', 10892: 'CWAP health', 10893: 'CWAP health', 10894: 'CWAP health', 
                        10900: 'CWAP status', 10901: 'CWAP status', 10902: 'CWAP status', 10903: 'CWAP status',
                        30100: 'AppCheck', 10811: 'Weight on wheels', 10812: 'Weight off wheels', 10957: 'Power on/off'}}
# replace eventId with readable values from the map
df_new.replace(eventId_map, inplace=True)
# in Python, it's a good practice to typecase categorical features to a category type to fasten up the processing of the data
df_new['eventId'] = df_new['eventId'].astype('category')
# define the overall SLA dataframe in which the overall availability will be stored
df_SLA = pd.DataFrame(columns=['date', 'server', 'tailsign', 'expected operation time', 'downtime', 'actual operation time', 'availability'])



## Extract grouped data

Some logged data comes as a concatenated string and, for further processing and better interpretation, now needs to be split and stored
in separate fields.

In [ ]:
# filter for eventId = 'AppCheck' and create a new column 'FlightNo' with the first element of the split column 'data.element0'
df_new['FlightNo'] = df_new[df_new['eventId'] == 'AppCheck']['data.element0'].str.split(';').str[0]
# filter for eventId = 'CWAP status' and create a new column 'PaxConnected' with the value from 'data.element5'
df_new['PaxConnected'] = df_new[df_new['eventId'] == 'CWAP status']['data.element5']
# filter for eventId = 'AppCheck' and take the fourth element out of 'data.element0'; get the first integer out of it and add this to new column 'PortalFrontendHomepage' (1 = success)
df_new['PortalFrontendHomepage'] = df_new[df_new['eventId'] == 'AppCheck']['data.element0'].str.split(';').str[3].str.split(':').str[1].str.split(',').str[0].astype(int)
# proceed with the other components that are checked by BCEL AppCheck
df_new['PortalFrontendApp'] = df_new[df_new['eventId'] == 'AppCheck']['data.element0'].str.split(';').str[4].str.split(':').str[1].str.split(',').str[0].astype(int)
df_new['PortalRuntime'] = df_new[df_new['eventId'] == 'AppCheck']['data.element0'].str.split(';').str[5].str.split(':').str[1].str.split(',').str[0].astype(int)
df_new['AnalyticsReceiver'] = df_new[df_new['eventId'] == 'AppCheck']['data.element0'].str.split(';').str[6].str.split(':').str[1].str.split(',').str[0].astype(int)
df_new['MovingMap'] = df_new[df_new['eventId'] == 'AppCheck']['data.element0'].str.split(';').str[7].str.split(':').str[1].str.split(',').str[0].astype(int)
df_new['FlightAPI'] = df_new[df_new['eventId'] == 'AppCheck']['data.element0'].str.split(';').str[8].str.split(':').str[1].str.split(',').str[0].astype(int)
df_new['InflightContentServer'] = df_new[df_new['eventId'] == 'AppCheck']['data.element0'].str.split(';').str[9].str.split(':').str[1].str.split(',').str[0].astype(int)
df_new['DRMWidevine'] = df_new[df_new['eventId'] == 'AppCheck']['data.element0'].str.split(';').str[10].str.split(':').str[1].str.split(',').str[0].astype(int)
df_new['DRMFairplay'] = df_new[df_new['eventId'] == 'AppCheck']['data.element0'].str.split(';').str[11].str.split(':').str[1].str.split(',').str[0].astype(int)

# extract power on/off timestamps to an explicit column and convert to datetime
df_new['PowerOn'] = df_new[df_new['eventId'] == 'Power on/off']['data.element0'].str.split(';').str[0]
df_new['PowerOn'] = pd.to_datetime(df_new['PowerOn'])
df_new['PowerOff'] = df_new[df_new['eventId'] == 'Power on/off']['data.element1'].str.split(';').str[0]
df_new['PowerOff'] = pd.to_datetime(df_new['PowerOff'])


## Configuration section

In this section we define the **components** that the Juypter Notebook shall consider for report generation.

**Hint**
> The AppCheck currently considers 9 different components to be mandatorily checked by it.
> These are:
> - Portal Frontend Homepage
> - Portal Frontend App
> - Portal Runtime
> - Analytics Receiver
> - Moving Map
> - Flight API
> - Inflight Content Server
> - DRM Widevine
> - DRM Fairplay

As per an agreement with the VA PDM from Dec, 2nd 2021, we consider the following components as relevant for measuring the BC IFE service availability:

> - Portal Frontend Homepage
> - Portal Frontend App
> - Portal Runtime
> - Flight API
> - Inflight Content Server

In [ ]:
# create a list of components for the diagram; we leave out the non-SLA-relevant components and combine PortalFrontendHomepage and PortalFrontendApp 
#components = ['PortalFrontendHomepage', 'PortalFrontendApp', 'PortalRuntime', 'AnalyticsReceiver', 'MovingMap', 'FlightAPI', 'InflightContentServer', 'DRMWidevine', 'DRMFairplay']
components = ['PortalFrontendHomepage', 'PortalFrontendApp', 'PortalRuntime', 'FlightAPI', 'InflightContentServer']
# create a list of all servers in the report 
servers = list()
for s in df_new['serialNumber'].unique().tolist():
    servers.append(s)
# if other servers (or a subset of servers) than contained in the CSV file shall be used, append those explicitly to the list
#servers.append('7CTCA20586')

month = '2021-11'
period = pd.Period(month, freq='M')
#df_SLA.date = pd.Series(pd.date_range(start=period.start_time, end=period.end_time, freq='D'))
month_days = pd.Series(pd.date_range(start=period.start_time, end=period.end_time, freq='D'))
# initialise the overall SLA dataframe
for day in month_days:
    # iterate over all servers
    for s in servers:
        # add a new row to df_SLA where df_SLA.date = day and df_SLA.server = s
        df_SLA = df_SLA.append(pd.DataFrame({'date': [day], 'server': [s], 'expected operation time': [0], 'downtime': [0], 'actual operation time': [0], 'availability': [0]}, index=[0]))
# create a duplicate of the column 'server', name it 'tailsign' and apply the tailsign_map to it
df_SLA['tailsign'] = df_SLA['server'].copy()
tailsign_map = {'tailsign': {'3442310010': 'VHYIO', '3467540006': 'VHYIR', '3352680006': 'VHYFJ', '3395600002': 'VHYIT', '3734240001': 'VHVUZ',
                             '3413940001': 'VHYFE', '3472530005': 'VHYFF', '3413940014': '', '7CTCA20586': 'VHYUD', '3413940006': 'VHYIJ'}}
df_SLA.replace(tailsign_map, inplace=True)


# Report visualisation

## Succeeded and failed AppChecks (optional) 

The subsequent charts show the totals for the succeeded and failed AppChecks on all defined servers in the time period  
that is contained in the CSV file.

(For further visualisations, refer to this)[https://towardsdatascience.com/stacked-bar-charts-with-pythons-matplotlib-f4020e4eb4a7]

In [ ]:
# create a working copy of df_new, filtered by eventId = 'AppCheck', using only the columns: timestamp, serialNumber, eventId, PortalFrontendHomepage
df_work_appcheck = df_new[df_new['eventId'] == 'AppCheck'][['timestamp', 'serialNumber', 'eventId', 'PortalFrontendHomepage', 'PortalFrontendApp', 'PortalRuntime', 'AnalyticsReceiver', 'MovingMap', 'FlightAPI', 'InflightContentServer', 'DRMWidevine', 'DRMFairplay']]
# define the labels to be used in the charts
labels = ['Portal Frontend', 'Portal Runtime', 'Flight API', 'Inflight Content Server']
# i is just a counter to print the respective dataframe df_list[i]
i = 0
# start defining the stacked bar plot
colors = ['#1D2F6F', '#8390FA', '#6EAF46', '#FAC748']
width = 0.35
#iterate over all servers and create a new dataframe for each server and component which contains the numbers of succeeded and failed component checks; store these new dataframes in df_list()
for s in servers:
    print('Iteration for server ' + str(s))
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.set_ylabel('Number of AppChecks')
    plt.xticks(
        rotation=45,
        horizontalalignment='right',
        fontweight='light',
        fontsize='medium',
    )
    # chart's title
    ax.set_title('AppChecks for Server ' + str(s))
    # suc is the series of successful appchecks for the current server and and the respective component
    suc = pd.Series(dtype = 'object')
    # fail is the series of failed appchecks for the current server and respective component
    fail = pd.Series(dtype = 'object')
    # create a list of dataframes
    df_list = list()
    # first, define a filter for all servers in list s
    df_server_filter = (df_new['serialNumber'] == s)
    # then, iterate over the labels (which are components of the appcheck) and create a dataframe per server and component
    for component in components:
        # reset the dataframe per server
        df_component = ()
        # define the components as categorical data using value 1, 0 (1 = success, 0 = failure)
        df_work_appcheck[component] = pd.Categorical(df_work_appcheck[component], categories=[1, 0], ordered=True)
        # create a new series for each component by grouping the filtered original dataframe by the component and count the number of occurences
        df_component = df_work_appcheck.loc[df_server_filter, :].groupby([component])[component].count()
        df_component = df_component.to_frame()
        # append df_component.loc[1] to suc; suc now contains the number of successful appchecks for the current server and respective component
        suc = suc.append(df_component.loc[1])
        # append df_component.loc[0] to fail; fail now contains the number of failed appchecks for the current server and respective component
        fail = fail.append(df_component.loc[0])
        df_list.append(df_component)
        i += 1

    #we don't want to differentiate Portal Frontend Homepage and Portal Frontend App but consolidate them as Portal Frontend in the report
    if suc.PortalFrontendApp < suc.PortalFrontendHomepage: 
        # drop suc.PortalFrontendHomepage
        suc = suc.drop(suc.index[suc.index == 'PortalFrontendHomepage'])
        fail = fail.drop(fail.index[fail.index == 'PortalFrontendHomepage'])
    else:
        # drop suc.PortalFrontendApp
        suc = suc.drop(suc.index[suc.index == 'PortalFrontendApp'])
        fail = fail.drop(fail.index[fail.index == 'PortalFrontendApp'])
    # create a matplotlib ax.bar with labels and the data from the list of dataframes
    if (suc.sum(axis=0) > 0 or fail.sum(axis=0) > 0): 
        ax.bar(labels, suc.to_list(), width, color=colors[3], label='Suceeded')
        ax.bar(labels, fail.to_list(), width, bottom=suc.to_list(), color=colors[0], label='Failed')
        for index,data in enumerate(suc.to_list()):
            plt.text(x=index , y =data/2 , s=f"{data}" , fontdict=dict(fontsize=10))
        for index,data in enumerate(fail.to_list()):
            plt.text(x=index , y =suc.to_list()[index]+10 , s=f"{data}" , fontdict=dict(fontsize=10))
        # remove spines
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        # adjust limits and draw grid lines
        plt.ylim(-0.5, ax.get_yticks()[-1] + 0.5)
        ax.set_axisbelow(True)
        ax.yaxis.grid(color='gray', linestyle='dashed')
        # legend
        plt.legend(['Success', 'Failed'], loc='upper left', ncol=4, frameon=False)
        plt.show()
    else:
        print('No appchecks for server ' + str(s))
        continue



# SLA report - operational time expected vs achieved per day and server

## Report definition
We create a monthly report which includes per day and server the 
- Expected Operation Time
- Sum of unplanned downtime
- Passenger Service Availability Time
- Passenger Service Availability Status % 

## Current assumptions
### Components which to consider
As earlier explained we only consider certain SW components to be relevant for the SLA
```
PortalFrontendHomepage, PortalFrontendApp (both in one consolidated status), PortalRuntime, Flight API, Inflight Content Server
```
From these components, we consider a failed AppCheck as a downtime of the Portal.  
Several failed AppChecks in a row, sum up to a higher downtime.
For instance: (AppCheck occurs every 10 minutes)
> Portal Frontend on server <server> failed **2 times** in a row, 
> the calculated downtime is **20 minutes** for this server.  
> Any other component failing in the same period of time won't add to the calculated downtime.  

~~In addition, we consider the CWAP status and health in a way that an error on the CWAPs also leads 
to a decreased Passenger Service Availability time.~~

> ~~A CWAP that shows an error in a single check is considered to cause a downtime 10 minutes on 25% of the passengers.~~  
> A CWAP that shows an error for one of the radios is not considered to cause a downtime since since rooming takes place  
> and passengers are connected to the next CWAP

### Expected Operational Time
- We consider every period of time between a 'power on' and 'power off' event that happens on-ground to add to the overall Expected Operational Time of the server. Power cycles that happen in-flight decrease the time (pls. refer to 'Additional considerations')

### Additional considerations
- Inflight (after Weight off wheels, before Weight on wheels), we consider a sequence of Power on/Power off events as a downtime of the service
- Inflight, we consider the lack of AppChecks as a service downtime (remember: we conduct AppChecks every 10 minutes; if no AppCheck occur on a 1h flight, we consider this as a downtime of 1 hour)


## Basic definitions

In [ ]:
# create a new working df and only copy the columns we need
#df_work_events = df_new[['timestamp', 'serialNumber', 'eventId', 'data.element0', 'data.element1', 'data.element2', 'data.element3', 'data.element4', 'data.element5', 'data.element6', 'FlightNo']]
df_work_events = df_new.copy()
# hey, let's make column 'timestamp' really a timestamp and use it as index
df_work_events['timestamp'] = pd.to_datetime(df_work_events['timestamp'], format='%b %d, %Y @ %H:%M:%S.%f')
#df_work_events.set_index('timestamp', inplace=True)

# filter_power_on is furthermore used to filter all Power on/off events as well as Weight on/off wheels to be able to show operation times between those events
# if operation time should be aggregated on a higher level, Weight on/off wheels events are not relevant and can be left ou
#filter_power_on = (df_work_events['eventId'] == 'Power on/off')
filter_power_on = (df_work_events['eventId'] == 'Power on/off') | (df_work_events['eventId'] == 'Weight off wheels') | (df_work_events['eventId'] == 'Weight on wheels')
filter_wow = (df_work_events['eventId'] == 'Weight off wheels') | (df_work_events['eventId'] == 'Weight on wheels')


## Calculate the expected operational time per server and date

Assumption: The expected operational time is the sum of time differences between a server's first log (using timestamp for it) and it's shutdown time (available in data.element1 if eventId == 'Power On/Off')

### Check on power on/off events that happen inflight and on-ground

Inflight := the time between a ```Weight off wheels``` and a ```Weight on Wheels``` event.  
All the time between a ```Weight off wheels``` and a ```Weight on Wheels``` event is considered to be Expected Operation Time.
A ```Power on/off```event that happens inflight decreases the overall operation time.

Even though that the SLA is defined on-ground as well as in-flight, we don't consider on-groud server re-boots as downtimes 
they could be caused by several reasons: LAME, Aircraft power change and others.

We calculate the downtime that happened inflight per server and day in the following procedure.  
Moreover, we calculate the time per server and day between each 'power on' and 'power off' event that happened on-ground.  
All these time differences, we sum up to be the Expected Operational Time per server and day.

In [ ]:
warnings.filterwarnings('ignore')
#let's restrict the list of servers to a single server (just for experimenting)
#servers = list()
#servers.append('3352680006')
#iterate over all servers in list servers
for s in servers:
    #some basic definitions and local variable declarations
    #from df_work_events, filter all logs for the currently processed server
    filter_server = (df_work_events['serialNumber'] == s)
    #create a new dataframe for the current server, with columns: timestamp, eventId, serialNumber, Power On, Power Off
    col = ['timestamp', 'eventId', 'serialNumber', 'PowerOn', 'PowerOff']
    df_server = df_work_events.loc[filter_server, :]
    #drop all columns except the ones we need (timestamp, eventId, serialNumber, Power On, Power Off)
    df_server = df_server[col]
    #make sure the timestamp is really a timestamp; PowerOn and PowerOff are already timestamps
    df_server['timestamp'] = pd.to_datetime(df_server['timestamp'])
    # add the columns 'downtime' and 'opstime' to the dataframe df_server, both of type datetime with value NaN
    df_server['downtime'] = pd.to_datetime(np.nan)
    df_server['opstime'] = pd.to_datetime(np.nan)
    # we now iterate over the WoW events and identify the Power on/off events that happened inflight/on the ground
    # the Power on/off events that happened inflight are considered as downtime
    # the Power on/off events that happened on the ground are considered as operation time
    filter_wow = (df_server['eventId'] == 'Weight off wheels') | (df_server['eventId'] == 'Weight on wheels')
    wow_timestamps = pd.Series(index=None, dtype='object')
    wow_timestamps = df_server[filter_wow]['timestamp']
    # - [ ] need to decide whether I keep the following line of code or remove it
    df_server.set_index('timestamp', inplace=True)
    #iterate over the pairs of WoW timestamps; so, in each case we should get two WoW events with other events between them
    for (index, ts) in enumerate(wow_timestamps):
        # if the current WoW event is the last one, we don't have a pair of WoW events to compare to
        if index < len(wow_timestamps) - 1:
            #let's find out whether we are inflight or on the-ground; the log data is sorted by timestamp descending
            wow_eventId = df_server.loc[ts, 'eventId']
            # get the current timestamp (WoW event) and the next timestamp (WoW event) to then filter all events in this WoW cycle
            current_ts = ts # still a timestamp
            next_ts = wow_timestamps.to_list()[index + 1] # still a timestamp
            #we consider all the time between the current and the next WoW event as opstime; only if we are on ground, we will reduce the opstime by the times the server had been shut down
            opstime = (current_ts - next_ts).total_seconds()
            # slice the data and extract all log rows between WoW current_ts and WoW next_ts
            df_server_wow_cycle = df_server.loc[current_ts:next_ts, :]
            # add the columns 'downtime' and 'opstime' to the dataframe df_server, both of type datetime
            #- [ ] need to decide whether I keep the following line of code or remove it
            #df_server_wow_cycle['downtime'] = pd.to_datetime(np.nan)
            #df_server_wow_cycle['opstime'] = pd.to_datetime(np.nan)
            # filter all 'Power on/off' events from dataframe
            filter_wow_cycle_power = (df_server_wow_cycle['eventId'] == 'Power on/off')
            #1st use case: Weight on Wheels: the past period was inflight. We consider all server reboot events as downtime
            if wow_eventId == 'Weight on wheels':
                # for all Power on/off events found, we calculate the downtime as the difference between the event's PowerOn- (in column 'PowerOn') and the PowerOff-timestamp (in column 'PowerOff')
                # we store the downtime in the column 'downtime' off the respective Power on/off event's row
                #Have we found some inflight Power cycles?
                if len(df_server_wow_cycle[filter_wow_cycle_power]) > 0:
                    #If yes...
                    #print('Server ' + str(s) + ' has ' + str(len(df_server_wow_cycle.loc[filter_wow_cycle_power, :])) + ' power on/off events in-flight at ' + str(current_ts))
                    for index, row in df_server_wow_cycle[filter_wow_cycle_power].iterrows():
                        # get the timestamp of the current Power on/off event
                        power_on_ts = row['PowerOn']
                        # get the timestamp of the last Power on/off event
                        power_off_ts = row['PowerOff']
                        # calculate the downtime as the difference between the current timestamp and the last timestamp in seconds
                        downtime = (power_on_ts - power_off_ts).total_seconds()
                        # store the downtime in the respective Power on/off event in the server's dataframe
                        df_server.loc[index, 'downtime'] = downtime
                        #print('Have added ' + str(df_server.loc[ts, 'downtime']) + ' seconds to the downtime of ' + str(s))
                #being inflight, we add to the Expected Operation Time the full time span between the two WoW events
                df_server.loc[current_ts, 'opstime'] = opstime
                #print('Have added ' + str(opstime) + ' seconds to the total opstime of ' + str(s))
            #2nd use case: Weight off Wheels: the past period was on the ground. We consider all server reboot events as operation time
            elif wow_eventId == 'Weight off wheels':
                #Have we found some on-ground Power cycles?
                if len(df_server_wow_cycle[filter_wow_cycle_power]) > 0:
                    #print('Server ' + str(s) + ' has ' + str(len(df_server_wow_cycle.loc[filter_wow_cycle_power, :])) + ' power on/off events on-ground at ' + str(current_ts))
                    #If yes...we will decrease the opstime by the time the server has been shut down. On ground, we don't consider these power cycles as unplanned downtime
                    #so they don't do anything to the downtime
                    for index, row in df_server_wow_cycle[filter_wow_cycle_power].iterrows():
                        # get the timestamp of the current Power on/off event
                        power_on_ts = row['PowerOn']
                        # get the timestamp of the last Power on/off event
                        power_off_ts = row['PowerOff']
                        # calculate the operation time as the difference between the current timestamp and the last timestamp in seconds
                        opstime = opstime - (power_on_ts - power_off_ts).total_seconds()
                df_server.loc[current_ts, 'opstime'] = opstime
                #print('Have added ' + str(opstime) + ' seconds to the total opstime of ' + str(s))
    # we now have the downtime and the operation time for the server
    # let's now group the downtime by date and sum up all the values in the column 'sum'
    df_server_grouped_downtime = df_server.groupby(df_server.index.date)[['downtime']].sum()
    for (index, row) in df_server_grouped_downtime.iterrows():
        #check if row[0] is NaN
        if not np.isnan(row[0]):
            df_SLA.loc[(df_SLA.server == s) & (df_SLA.date.dt.date == index), 'downtime'] = row['downtime']
        else:
            df_SLA.loc[(df_SLA.server == s) & (df_SLA.date.dt.date == index), 'downtime'] = 0
    df_server_grouped_opstime = df_server.groupby(df_server.index.date)[['opstime']].sum()
    for (index, row) in df_server_grouped_opstime.iterrows():
        #check if row[0] is NaN
        if not np.isnan(row[0]):
            df_SLA.loc[((df_SLA.server == s) & (df_SLA.date.dt.date == index)), 'expected operation time'] = row['opstime']
        else:
            df_SLA.loc[((df_SLA.server == s) & (df_SLA.date.dt.date == index)), 'expected operation time'] = 0





### Analysis of the aggregated AppCheck

In the subsequent analysis, we check how many appchecks succeeded or failed between two Power on/off events for the defined servers.  

We can easily increase transparency for this analysis and consider 'Weight on/off wheels' as well  
(by changing the filter *filter_power_on*) and thus, come to a view where we can differentiate  
in-flight and on-ground behaviour.

The result of this report currently contains:  
- The event (Power on/off, WoW) which occured (event) and 
- when server has been powered on and off (poweron, poweroff),
- whether the CWAPs have been successfully activated (status: *Activated* in column cwap) in the period between the previous and the current event
- how many clients have been connected (client connected) in this period
- the number of appchecks having been conducted (appcheck) in this period
- the calculated availability (availability) in terms of succeeded or failed appchecks in this period
- the ops time (opstime) of the server as the difference between the latest power off event and the last power on event

## the following section needs to be continued......

In [63]:
# iterate over all servers in list servers
for s in servers:
    #some basic definitions and local variable declarations
    downtime = 0
    #from df_work_events, filter all logs for the currently processed server
    filter_server = (df_work_events['serialNumber'] == s)
    #create a new dataframe for the appcheck analysis of the current server, with columns: timestamp, eventId, serialNumber, Power On, Power Off
    col = ['timestamp', 'eventId', 'data.element4', 'data.element5', 'serialNumber', 'PaxConnected'] + components
    df_appcheck = df_work_events.loc[filter_server, :]
    #drop all columns except the ones we need (timestamp, eventId, serialNumber, Power On, Power Off)
    df_appcheck = df_appcheck[col]
    #make sure the timestamp is really a timestamp;
    df_appcheck['timestamp'] = pd.to_datetime(df_appcheck['timestamp'])
    # add the column 'downtime' to the dataframe (of type datetime with value NaN)
    df_appcheck['downtime'] = pd.to_datetime(np.nan)
    #filter df_appcheck for all entries with eventId = 'AppCheck'; these are the events we are interested in
    filter_appcheck = (df_appcheck['eventId'] == 'AppCheck')
    #iterate over all appcheck events
    #to iterate over all WoW events, we need an index
    wow_event_index = 0
    #I want to have all WoW events for the current server in a single dataframe
    filter_appcheck_wow = ((df_appcheck['eventId'] == 'Weight off wheels') | (df_appcheck['eventId'] == 'Weight on wheels'))
    df_appcheck_wow = df_appcheck.loc[filter_appcheck_wow, :]
    #create a view/filter on appcheck_timestamps to iterate over all timestamps
    appcheck_timestamps = pd.Series(index=None, dtype='object')
    appcheck_timestamps = df_appcheck[filter_appcheck]['timestamp']
    for index, ts in enumerate(appcheck_timestamps):
        if index < len(appcheck_timestamps) - 1:
            if index == 431:
                print('Here we are')
            # get the timestamp of the current appcheck event
            # - [ ] make sure not to iterate over the wow-events series end
            current_ts = ts
            # get the timestamp of the next appcheck event to calculate the max downtime
            next_ts = appcheck_timestamps.to_list()[index + 1]
            #look back in history and find the previous WoW event; depending on the WoW event, we will add to the downtime or not
            #if there is no further WoW event or no WoW event at all, we take the last one and derive from it in which flight phase we are
            #check whether df_app_wow_cycle has any entries or if wow_event_index is greater than the amount of entries in df_appcheck_wow
            if (len(df_appcheck_wow) > 0):# and (index < (len(df_appcheck_wow) - 1)):
                #define variable wow_event and store the value of the previous WoW event 
                #from df_appcheck, get the first element for which timestamp < current_ts
                wow_event = df_appcheck_wow.loc[df_appcheck_wow['timestamp'] < current_ts, :].iloc[0, :]['eventId']
                #wow_event = df_appcheck_wow.iloc[wow_event_index]['eventId']
                #define variable wow_timestamp and store the timestamp of the previous WoW event
                wow_timestamp = df_appcheck_wow.loc[df_appcheck_wow['timestamp'] < current_ts, :].iloc[0, :]['timestamp']
                #wow_timestamp = df_appcheck_wow.iloc[wow_event_index]['timestamp']
                wow_event_index = wow_event_index + 1
                appcheck_succeeded = df_appcheck.loc[df_appcheck['timestamp'] == current_ts, components].sum(axis=1)
                if appcheck_succeeded.sum() < 5:
                    print('Hey, found an appcheck issue for ' + str(s) + ' at ' + str(current_ts))
                #if wow_event is Weight off wheels (we are inflight!), we have to calculate the downtime as the difference between the current timestamp and the last timestamp in seconds
                if wow_event == 'Weight off wheels':
                    #have we had a Portal downtime? (i.e. one of the components' appchecks has been 0 and thus the sum of the appchecks is smaller than the sum of the components)
                    appcheck_succeeded = df_appcheck.loc[df_appcheck['timestamp'] == current_ts, components].sum(axis=1)
                    if (not appcheck_succeeded.empty) and (appcheck_succeeded.iloc[0] < len(components)):
                        #calculate the downtime as the difference between the current timestamp and the last timestamp in seconds
                        downtime = (current_ts - next_ts).total_seconds()
                        #store the downtime in the respective appcheck event in the server's dataframe
                        df_appcheck.loc[ts, 'downtime'] = downtime
                #else: # if wow_event == 'Weight on wheels':
                    # do nothing since we have been on ground
    # - [ ] aggregate the downtime for each day

        

Hey, found an appcheck issue for 3442310010 at 2021-12-01 04:32:45.466000
Hey, found an appcheck issue for 3442310010 at 2021-12-01 04:32:45.447000
Hey, found an appcheck issue for 3442310010 at 2021-11-27 06:41:46.733000


TypeError: int() argument must be a string, a bytes-like object or a number, not 'Timestamp'

In [ ]:
# iterate over all servers in list servers
for s in servers:
    print('Iteration for server ' + str(s))
    server_downtime = 0
    filter_server = (df_work_events['serialNumber'] == s)
    #filter_power_on is defined in a section earlier in this notebook (!!! it might include power events only or WoW events as well!!)
    filter = (filter_server) & (filter_power_on)
    # define a temporary dataframe for each server, on which we perform date-based selections
    df_appcheck = df_work_events.loc[filter_server, :]
    # define a pd Series that will store all timestamps when event 'Power on/off' was logged on this server
    power_timestamps = pd.Series(dtype='object');
    # create a series for each server with the timestamp of match for eventId = 'Power on/off'
    power_timestamps = df_appcheck[filter_power_on]['timestamp']
    # make timestamp the index, so that we can localise certain rows easily
    df_appcheck.set_index('timestamp', inplace=True)
    # create a new dataframe df_sla with timestamp as index and the columns: event, timestamp, cwap, appcheck, availability 
    df_sla = pd.DataFrame(columns=['event', 'poweron', 'poweroff', 'timestamp', 'cwap', 'client connected', 'appcheck', 'availability', 'powercycle'])
    # fill the dataframe with the values from the series
    # first, fill in when Power on/off occured
    df_sla['event'] = df_work_events[filter]['eventId']
    df_sla['timestamp'] = power_timestamps
    # data.element0 contains the power on timestamp and data.element1 contains the power off timestamp
    df_sla['poweron'] = df_work_events[filter]['data.element0']
    df_sla['poweroff'] = df_work_events[filter]['data.element1']
    df_sla['poweron'] = pd.to_datetime(df_sla['poweron'])
    df_sla['poweroff'] = pd.to_datetime(df_sla['poweroff'])
    # second, from df_work_events filter all events 'CWAP status' between two power cycles and fill in the dataframe
    # iterate over the pairs of elements of the series power_timestamps (thus, the period between two power cycles)
    for (index, ts) in enumerate(power_timestamps):
        if index < len(power_timestamps.to_list()) - 1:
            # get the current timestamp (start powercycle) and the next timestamp (end powercycle) to then filter all events in this power cycle
            current_ts = ts
            next_ts = power_timestamps.to_list()[index + 1]
            #from df_temp slice all rows between current_ts and next_ts to be able to get all events for one power cycle
            df_temp_slice = df_appcheck.loc[current_ts:next_ts, :]
            # make columns data.element5 and data.element6 from df_temp_slice integers and ignore NaN values
            df_temp_slice['data.element5'] = pd.to_numeric(df_temp_slice['data.element5'], errors='coerce')
            df_temp_slice['data.element6'] = pd.to_numeric(df_temp_slice['data.element6'], errors='coerce')
            # collect and fill in the connected pax for this power cycle
            # define a list of columns in which connected pax numbers are stored
            pax_connected_list = ['data.element5', 'data.element6']
            # sum up the values from columns ['data.element5', 'data.element6'] in df_temp_slice, where eventId = 'CWAP status'
            pax_connected = df_temp_slice.loc[df_temp_slice['eventId'] == 'CWAP status', pax_connected_list].sum(axis=1)
            # if pax_connected isn't empty and the sum of axis=1 is not 0, then fill in the value
            if not pax_connected.empty and pax_connected.sum(axis=0) != 0:
                df_sla.loc[df_sla.timestamp == current_ts, 'client connected'] =  pax_connected.sum(axis=0)
            else:
                df_sla.loc[df_sla.timestamp == current_ts, 'client connected'] =  0
            # in the next step, aggregate the status of the appcheck for this power cycle for the defined components in list 'components'
            # succeeded appchecks per component are written as 1 in the respective component's column
            # we sum up the values (1 or 0) from the components' columns per appcheck conducted;
            appcheck_succeeded = df_temp_slice.loc[df_temp_slice['eventId'] == 'AppCheck', components].sum(axis=1)
            # if at least one component appcheck failed, we add 10 minutes (and only 10!) to the current server's downtime
            if not appcheck_succeeded.empty: # and appcheck_succeeded < len(components):
                #iterate over all elements in series appcheck_succeeded
                for (index, succeeded) in enumerate(appcheck_succeeded):
                    if succeeded < len(components):
                        server_downtime = server_downtime + 10*60
                        #add the server_downtime directly to df_SLA with server == s and date == date(current_ts)
                        date = current_ts.date()
                        try:
                            server_downtime += df_SLA.loc[(df_SLA.server == s) & (df_SLA.date == date), 'downtime'].values[0]
                        except:
                            server_downtime += 0
                        finally:
                            df_SLA.loc[(df_SLA.server == s) & (df_SLA.date.dt.date == date), 'downtime'] = server_downtime
            # define appchecks as the number of rows from df_temp_slice where eventId = 'AppCheck'; i.e. appchecks is the number of appchecks conducted in this power cycle
            appchecks = df_temp_slice.loc[df_temp_slice['eventId'] == 'AppCheck'].shape[0]
            # we now sum up the overall number of appchecks succeded in this powercycle
            sum_appcheck_succeeded = appcheck_succeeded.sum(axis=0)
            # avail is the number of succeeded component appchecks divided by the number of appchecks (multiplied by number of components) multiplied by 100
            component_appchecks = appchecks * len(components)
            avail = (sum_appcheck_succeeded / (component_appchecks)) * 100
            # finally, write the calculated values to the dataframe
            df_sla.loc[df_sla.timestamp == current_ts, 'appcheck'] =  sum_appcheck_succeeded
            df_sla.loc[df_sla.timestamp == current_ts, 'availability'] =  avail
            # finally, aggregate the cwap health status for this power cycle
            # define a list of columns in which cwap status is stored; data.element3 and 4 hold the actual radio status 
            cwap_list = ['data.element3', 'data.element4']
            # filter for CWAP health status 'Error' and if such a status occured, write it to the dataframe and add 10 minutes to the current server's downtime
            cwap_health_filter = (df_temp_slice['eventId'] == 'CWAP health') & ((df_temp_slice['data.element3'] == 'Error') | (df_temp_slice['data.element4'] == 'Error'))
            # sum up the values from columns in cwap_list, where eventId = 'CWAP health' to detect whether one of the radios is in error
            cwap = df_temp_slice.loc[cwap_health_filter, cwap_list].count(axis=1)
            if not cwap.empty and cwap.sum(axis=0) != 0:
                cwap_down = cwap.sum(axis=0) / 2
                df_sla.loc[df_sla.timestamp == current_ts, 'cwap'] =  'Error'
                ####################################################
                # as per a common decision on Dec, 9th, we won't consider CWAP reported errrors as service downtimes
                ####################################################
                #server_downtime = server_downtime + cwap_down*10*60
            else:
                df_sla.loc[df_sla.timestamp == current_ts, 'cwap'] =  'Activated'
            # last but not least, fill in the powercycle column; it's the time difference between the poweron and poweroff of the current powercycle
            # if poweron is not empty or NaN and poweroff is not empty or NaN, then fill in the value
            power_on = df_sla.loc[df_sla.timestamp == current_ts, 'poweron'].count()
            power_off = df_sla.loc[df_sla.timestamp == current_ts, 'poweroff'].count()
            if (power_on > 0) and (power_off > 0):
                df_sla.loc[df_sla.timestamp == current_ts, 'powercycle'] =  df_sla.loc[df_sla.timestamp == current_ts, 'poweroff'] - df_sla.loc[df_sla.timestamp == current_ts, 'poweron']
            df_sla['powercycle'] = pd.to_timedelta(df_sla['powercycle'],'h')
            df_sla['ops time'] = df_sla.poweron - df_sla.poweroff
            #df_sla['powercycle'] = df_sla['powercycle']/np.timedelta64(1,'h')

        # remember, the index of series power_timestamps is the same as in df_work_events
    print('Downtime for server ' + s + ': ' + str(server_downtime))
    df_sla.set_index('timestamp', inplace=True)
#    df_sla.to_markdown('sla_' + str(s) + '.md')

# Final calculations of actual operational time and availability


In [ ]:
# fill in the actual operation time which is the 'expected' operation time minus the downtime
# if you re-run solely this section, you will receive an error, since the operation performed will fail after column has been converted to datetime some lines below
# fill in availability, which is the 'expected operation time' minus the downtime divided by the 'expected operation time' and take care about divisions by zero
try:
    eot_filter = df_SLA['expected operation time'] != 0
    df_SLA.loc[eot_filter, 'actual operation time'] = (df_SLA.loc[eot_filter , 'expected operation time'] - df_SLA.loc[eot_filter, 'downtime'])
    df_SLA.loc[eot_filter, 'availability'] = ((df_SLA.loc[eot_filter , 'expected operation time'] - df_SLA.loc[eot_filter, 'downtime']) / df_SLA.loc[eot_filter, 'expected operation time']) * 100
except:
    df_SLA['availability'] = 0
# convert the columns 'expected operation time', 'downtime' and 'acutal operation time' which currently contain a number in seconds, to hours and format it HH:MM:SS
df_SLA['actual operation time'] = (pd.to_datetime(df_SLA['actual operation time'], unit='s')).dt.strftime('%H:%M:%S')
df_SLA['expected operation time'] = (pd.to_datetime(df_SLA['expected operation time'], unit='s')).dt.strftime('%H:%M:%S')
df_SLA['downtime'] = (pd.to_datetime(df_SLA['downtime'], unit='s')).dt.strftime('%H:%M:%S')
#format column 'availability' as a percentage number
df_SLA['availability'] = df_SLA['availability'].map('{:,.2f}%'.format)
# show the column types of df_SLA
print(df_SLA.dtypes)

In [ ]:
df_SLA['availability'] = df_SLA['availability'].map('{:,.2f}%'.format)

In [ ]:
# write the dataframe df_SLA to a csv file
df_SLA.to_csv('sla_' + str(s) + '.csv')